## Introduction

The goal of this assignment is to program the STM32 microcontroller to run a simple frequency detection algorithm. The algorithm in question will perform frequency detecion in 3 steps:
1. Compute the real-valued fast fourier transform ([`rfft`](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html)) of the sampled input signal.
    * The `rfft` function exploits the symmetric nature of a real-valued signal's DFT, so it does not have to perform quite as many calculations. The output is limited to *only* the positive half of the frequency spectrum, meaning that for $N$ signal samples the `rfft` output has length $\frac{N}{2} + 1$.
2. Identify the highest peak location in the `rfft` output.
3. Convert peak location to wave frequency measured in $\text{Hz}$, and present this value as system output.

Using python, the implementation of this algorithm is fairly straightforward:


In [6]:
import numpy as np
def freq_detection(x_n: np.ndarray, fs: int)->float:
    """
    Identifies the primary sinusoidal component in a signal x[n]
    by calculting the DFT of x[n], and selecting the frequency 
    component with the highest magnitude. 

    Parameters:
    x_n - signal samples x[n] to be analyzed
    fs - sampling frequency

    Returns:
    freq - Frequency of the strongest frequency component measured in Hz
    """
    N = len(x_n)
    X_m = np.fft.rfft(x_n)
    m_peak = np.argmax(np.abs(X_m))
    freq = m_peak/N*fs
    return freq

So far, the task seems pretty uncomplicated: $N$ samples of a signal goes in, the rfft is calculated, some basic peak-finding is used, and out comes the signal's frequency. The main problem we need to solve however, is the following:

> How do we implement an algorithm which processes a digital signal $x[n]$ in segments of $N$ samples at a time on a microcontroller which is acquiring new samples in real-time?

The solution to this problem is to create a circular buffer which the ADC can write it's samples to, and which will always contain a log of the $N$ most recent samples acquired. Once all the samples in the buffer have been replaced, we can feed the entire buffer into our frequency detection algorithm and calculate a frequency estimate which will update every $\Delta t = \frac{f_s}{N}$ seconds.

This, at least, is the general shape of the solution. However, the RFFT calculation will take some time, and if the ADC continues writing samples to the same buffer that is used as RFFT input while the calculations are underway, we may get some rather problematic errors in our output. In fact, we will have to set up a [double buffer](https://en.wikipedia.org/wiki/Multiple_buffering) to ensure that the buffer being overwritten by the ADC is not the same as the buffer being fed into our frequency detection algorithm. 

The simplest way to implement double buffering is to create one big array of length $N\cdot 2$ to act as both buffers. We can then set up interrupt service routines (ISR) to trigger both when the buffer array is half way full and when it is entirely full. Frequency detection is then accomplished by feeding alternating halves of the buffer array into our frequency detection algorithm. 

![Dual Buffer Illustration](Figures/DualBufferAnim.gif)

Programming the microcontroller to perform this function is a far cry from being trivial, but following the steps laid out in this assignment should lead you to a working solution. 

|Task | Topic |
|---|---|
|[Step 1](1_DSP_Library.ipynb)| Getting started with the CMSIS-DSP library. |
|[Step 2](2_Device_config.ipynb)| Configuring peripherals.|
|[Step 3](3_Buffer_management.ipynb)| Managing the buffers in `C`.|
|[Step 4](4_RFFT_on_STM32.ipynb)| Performing the RFFT.|